# Concate Discrete Points

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import os

In [19]:
df = pd.read_csv('../dataset/tracks/PETS09_1.csv', header=None)
df.columns = ['frame_id','track_id','xmin','ymin','width','height', 
              'confidence','neg_1', 'neg_2', 'neg_3']
df.index = df['frame_id']
df = df.drop(['frame_id'], axis=1)
df.head()

,track_id,xmin,ymin,width,height,confidence,neg_1,neg_2,neg_3
frame_id,,,,,,,,,
5,1,602.36,239.12,43.60,84.11,1,-1,-1,-1
5,2,492.69,159.38,29.19,78.63,1,-1,-1,-1
5,3,281.47,211.68,30.21,88.18,1,-1,-1,-1
6,1,595.00,239.77,43.72,84.01,1,-1,-1,-1
6,2,489.06,160.83,28.98,78.20,1,-1,-1,-1


In [20]:
def join(df_track):
    prev_frame_idx = min(df_track['track_id'].index)
    results = []
    for frame_idx, currrent_row in df_track.iterrows():
        gap = frame_idx - prev_frame_idx
        if(gap > 1):
            results.append(str(prev_frame_idx)+' -> '+ str(frame_idx))
            currrent_row = np.array(currrent_row)
            previous_row = np.array(df_track.loc[prev_frame_idx].as_matrix())
            steps = (currrent_row - previous_row) / gap

            for i, frame in enumerate(range(prev_frame_idx+1,frame_idx)):
                df_track.loc[frame] = np.array(previous_row + (i+1) * steps).astype(int)

        prev_frame_idx = frame_idx
    df_track = df_track.sort_index()

    misses = np.squeeze(list(set(range(min(df_track.index), 
                                       max(df_track.index) + 1)).difference(df_track.index)))
    if(len(misses)==0 and len(results) > 0):
        print('Track:', int(df_track['track_id'].iloc[0]),', concatenation complete')
        print(results,'\n')
    elif(len(misses)!=0):
        print('Warning!! Frame:', int(df_track['track_id'].iloc[0]), ', concatenation incomplete\n')
    return df_track

In [21]:
concat = []

From, To = min(df['track_id']), max(df['track_id'])+1
for track_id in range(From, To):
    concat.append(join(df.loc[df['track_id']==track_id].copy()))
    
df_concat = pd.concat(concat)
df_concat = df_concat.sort_index()

Track: 1 , concatenation complete
['38 -> 40', '47 -> 55', '63 -> 72', '75 -> 89', '90 -> 92', '93 -> 108', '110 -> 112'] 

Track: 3 , concatenation complete
['31 -> 33', '85 -> 106'] 

Track: 4 , concatenation complete
['163 -> 168', '374 -> 422', '475 -> 483', '581 -> 609', '660 -> 663', '687 -> 689', '692 -> 694', '763 -> 765'] 

Track: 5 , concatenation complete
['158 -> 164'] 

Track: 6 , concatenation complete
['41 -> 43', '44 -> 58', '63 -> 84', '168 -> 171', '319 -> 326', '345 -> 348'] 

Track: 8 , concatenation complete
['211 -> 218', '339 -> 343'] 

Track: 11 , concatenation complete
['151 -> 156'] 

Track: 13 , concatenation complete
['163 -> 166', '211 -> 228', '279 -> 281'] 

Track: 14 , concatenation complete
['271 -> 276', '295 -> 297', '357 -> 360', '462 -> 465', '484 -> 487', '505 -> 509', '624 -> 628', '726 -> 732', '743 -> 748'] 

Track: 15 , concatenation complete
['498 -> 501', '522 -> 571', '629 -> 634', '671 -> 674', '749 -> 751'] 

Track: 17 , concatenation comp

In [22]:
df_concat.head()

,track_id,xmin,ymin,width,height,confidence,neg_1,neg_2,neg_3
frame_id,,,,,,,,,
5,1,602.36,239.12,43.60,84.11,1,-1,-1,-1
5,3,281.47,211.68,30.21,88.18,1,-1,-1,-1
5,2,492.69,159.38,29.19,78.63,1,-1,-1,-1
6,1,595.00,239.77,43.72,84.01,1,-1,-1,-1
6,2,489.06,160.83,28.98,78.20,1,-1,-1,-1


In [24]:
df_concat.to_csv('concat.csv', header=None)